# Data Explore

In [122]:
import os
import ee
import geemap.core as geemap

import pandas as pd
import geopandas as gpd

In [123]:
import sys
sys.path.append('..')
from libs.sentinel import mask_s2_clouds

In [124]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

# Set up GEE API
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID) #<- Remember to change this to your own project's name!

# Load data

In [125]:
directory = 'data/NZ_landslide_inventory/'
auckland_filepath = f'{directory}/cleaned_auckland.gpkg'
gdf = gpd.read_file(auckland_filepath, layer='landslides')
gdf[gdf['dateoccurence'].notnull()]

,GlobalID,latitude,longitude,region,dateoccurence,closestdate,geometry
0,cb20f8d7-abeb-4ce5-bafa-a75c01b03136,-36.931306,174.701880,Auckland Region,2023-02-13 23:00:00,2023-02-13 23:00:00,POINT (174.70188 -36.93131)
1,9da581bc-ff20-454f-b6fb-721279cd347a,-36.924269,174.705747,Auckland Region,2023-01-30 23:00:00,2023-01-30 23:00:00,POINT (174.70575 -36.92427)
2,2a361da4-7adf-4f4d-b94c-f3cf9ccccc46,-36.867368,174.738318,Auckland Region,2023-01-31 23:00:00,2023-01-31 23:00:00,POINT (174.73832 -36.86737)
3,4470b30b-6c80-46be-b0f7-e4ec0a5d72fd,-36.861593,174.655298,Auckland Region,2023-02-13 23:00:00,2023-02-13 23:00:00,POINT (174.6553 -36.86159)
4,cfcff56a-7cc6-41f5-9e7e-8da0daa5dbde,-36.817738,174.728127,Auckland Region,2023-01-31 23:00:00,2023-01-31 23:00:00,POINT (174.72813 -36.81774)
...,...,...,...,...,...,...,...
11911,8ac9ffa6-d365-4cb0-bc15-e54c10a63d8b,-36.807419,174.690856,Auckland Region,2023-01-26 11:00:00,2023-01-26 11:00:00,POINT (174.69086 -36.80742)
11912,2801a385-dc6a-44b3-9355-64140b2f1929,-37.133864,174.706530,Auckland Region,2023-01-26 11:00:00,2023-01-26 11:00:00,POINT (174.70653 -37.13386)
11913,c8cc9f20-efac-41b4-bf62-55a625a2ab6e,-36.938192,174.629223,Auckland Region,2023-01-26 11:00:00,2023-01-26 11:00:00,POINT (174.62922 -36.93819)
11914,fa946895-4967-4a14-9b59-48dd0898766e,-36.944683,174.663084,Auckland Region,2023-01-27 11:00:00,2023-01-27 08:00:00,POINT (174.66308 -36.94468)


In [126]:
gdf_jan = gdf[(gdf['dateoccurence'] >= '2023-01-01') & (gdf['dateoccurence'] < '2023-02-28')]
jan_gdf = gdf_jan.reset_index(drop=True)

In [127]:
gdf_jan

,GlobalID,latitude,longitude,region,dateoccurence,closestdate,geometry
0,cb20f8d7-abeb-4ce5-bafa-a75c01b03136,-36.931306,174.701880,Auckland Region,2023-02-13 23:00:00,2023-02-13 23:00:00,POINT (174.70188 -36.93131)
1,9da581bc-ff20-454f-b6fb-721279cd347a,-36.924269,174.705747,Auckland Region,2023-01-30 23:00:00,2023-01-30 23:00:00,POINT (174.70575 -36.92427)
2,2a361da4-7adf-4f4d-b94c-f3cf9ccccc46,-36.867368,174.738318,Auckland Region,2023-01-31 23:00:00,2023-01-31 23:00:00,POINT (174.73832 -36.86737)
3,4470b30b-6c80-46be-b0f7-e4ec0a5d72fd,-36.861593,174.655298,Auckland Region,2023-02-13 23:00:00,2023-02-13 23:00:00,POINT (174.6553 -36.86159)
4,cfcff56a-7cc6-41f5-9e7e-8da0daa5dbde,-36.817738,174.728127,Auckland Region,2023-01-31 23:00:00,2023-01-31 23:00:00,POINT (174.72813 -36.81774)
...,...,...,...,...,...,...,...
11911,8ac9ffa6-d365-4cb0-bc15-e54c10a63d8b,-36.807419,174.690856,Auckland Region,2023-01-26 11:00:00,2023-01-26 11:00:00,POINT (174.69086 -36.80742)
11912,2801a385-dc6a-44b3-9355-64140b2f1929,-37.133864,174.706530,Auckland Region,2023-01-26 11:00:00,2023-01-26 11:00:00,POINT (174.70653 -37.13386)
11913,c8cc9f20-efac-41b4-bf62-55a625a2ab6e,-36.938192,174.629223,Auckland Region,2023-01-26 11:00:00,2023-01-26 11:00:00,POINT (174.62922 -36.93819)
11914,fa946895-4967-4a14-9b59-48dd0898766e,-36.944683,174.663084,Auckland Region,2023-01-27 11:00:00,2023-01-27 08:00:00,POINT (174.66308 -36.94468)


In [144]:
idx = 0
event = gdf_jan.iloc[idx]
point = event.geometry.centroid
points = gdf_jan.geometry.centroid
start_date = gdf_jan['dateoccurence'].min().date().strftime("%Y-%m-%d")
end_date = ( gdf_jan['dateoccurence'].min().date() + pd.Timedelta(days=91)).strftime("%Y-%m-%d")
point = ee.Geometry.Point(point.x, point.y)

In [145]:
start_date, end_date

('2023-01-26', '2023-04-27')

In [146]:
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

In [147]:
# Convert GeoSeries to a list of ee.Geometry.Point
ee_points = [ee.Geometry.Point([p.x, p.y]) for p in points]

# Create an ee.FeatureCollection from the points
ee_features = [ee.Feature(p) for p in ee_points]
ee_feature_collection = ee.FeatureCollection(ee_features)

In [149]:
m = geemap.Map()
m.set_center(point.coordinates().get(0).getInfo(), point.coordinates().get(1).getInfo(), 15)
m.add_layer(dataset.min(), visualization, 'RGB')
# m.add_layer(point, {'color': 'red'}, 'My Pinpoint')
# Add the FeatureCollection to the map
m.addLayer(ee_feature_collection, {'color': 'red'}, 'Landslide Points')

m

Map(center=[-36.931306466746, 174.701880487593], controls=(WidgetControl(options=['position', 'transparent_bg'…